In [1]:
import sys, time, wmi, psutil
SYSTEM_INFO = wmi.WMI().Win32_OperatingSystem()[0]
"system: {0}, {1}, {2}".format(SYSTEM_INFO.Caption, SYSTEM_INFO.BuildNumber, SYSTEM_INFO.OSArchitecture) 
"memory: {}G".format(round(psutil.virtual_memory().total / 1024**3, 2))
"cpu: {}".format(psutil.cpu_count())
"python: {}".format(sys.version)
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))

'system: Microsoft Windows 10 教育版, 18363, 64 位'

'memory: 15.86G'

'cpu: 4'

'python: 3.7.1 (default, Oct 28 2018, 08:39:03) [MSC v.1912 64 bit (AMD64)]'

'2020-08-17 10:12:58'

- **@author**: run_walker
- **@references**:
    1. [数据结构--并查集的原理及实现](https://www.cnblogs.com/hapjin/p/5478352.html)
    2. [数据结构4——并查集（入门）](https://www.cnblogs.com/xzxl/p/7226557.html)
    3. [并查集（进阶）](https://www.cnblogs.com/xzxl/p/7341536.html)
    4. [ackerman函数](https://baike.baidu.com/item/ackerman%E5%87%BD%E6%95%B0/2750194?fr=aladdin)

# 数据结构
1. 一簇点可以依据相互间的连通性（无向）被分为一个个**连通子集**，为每个点都记录一个**父元**。一个连通子集内，存在且仅存在一个点，其父元为其自身，这个点被称为该连通子集的**代表元**，其他点的父元为代表元（或者继续向上追溯父元，最终仍为该代表元）。
2. 合并两个不连通的子集$A$和$B$时（在两个不连通的点间添加一条边），只需要将$A$的代表元$a$的父元赋为$B$的代表元$b$即可（反过来也可以）。
3. 查找一个元素的代表元的同时进行**路径压缩**，将所有中间元的父元都赋为代表元。

# 可视化

In [2]:
from IPython.display import IFrame

IFrame("https://visualgo.net/zh/ufds", "100%", 400)

# python实现

## 初步实现（unused）

In [37]:
from collections import defaultdict


class DisjointSet(dict):
    """并查集"""
    def __init__(self, iters):
        """
        初始化，每个点的父元都赋为自身
        :param iters:
        """
        for x in iters:
            self[x] = x
            
    def union(self, items):
        """
        将一簇点连通起来
        :param items:
        :return:
        """
        items = list(items)
        f = self.find(items[0])
        for item in items[1:]:
            itemf = self.find(item)
            if itemf != f:
                self[itemf] = f 

    def find(self, item):
        """
        查找代表元，同时进行路径压缩
        :param item:
        :return:
        """
        if self[item] != item:
            self[item] = self.find(self[item])  # 若 a -> b -> c -> 代表元x，则将a、b、c的父元均修改为x
        return self[item]
    
    def is_connected(self, u, v):
        """
        判断两个点是否连通
        :param u:
        :param v:
        :return:
        """
        return self.find(u) == self.find(v)

    def get_cluster(self, item=None):
        """
        获得指定元素的所在的连通子集，如无指定，则统计所有连通子集
        :return: 
        """
        if item is None:
            clusters = defaultdict(set)
            for item in self:
                clusters[self.find(item)].add(item)
            return clusters
        else:
            sets = set()
            itemf = self.find(item)
            for k in self:
                if self.find(k) == itemf:
                    sets.add(k)
            return sets
    
    def count(self):
        """返回连通子集个数"""
        return len(self.get_cluster())

In [38]:
d = DisjointSet(range(13))
d.count()

13

In [39]:
sets = [{0, 1, 2}, {3, 4}, {5, 6, 7}, {8, 9, 10, 11}, {12}]
for items in sets:
    d.union(items)

d 

{0: 0,
 1: 0,
 2: 0,
 3: 3,
 4: 3,
 5: 5,
 6: 5,
 7: 5,
 8: 8,
 9: 8,
 10: 8,
 11: 8,
 12: 12}

In [40]:
d.count()

5

In [41]:
d.find(11)

d.is_connected(0, 1)
d.is_connected(0, 3)

d.get_cluster()

d.get_cluster(2)

8

True

False

defaultdict(set,
            {0: {0, 1, 2},
             3: {3, 4},
             5: {5, 6, 7},
             8: {8, 9, 10, 11},
             12: {12}})

{0, 1, 2}

## 优化

### 去除合并中的随机性（unused）
初步实现中合并两个连通子集时是随机修改代表元的父元的，当待合并的两个连通子集作为树（连边前对代表元是否相等的检查，保证了不会生成环）的深度差异很大时，如果碰巧将深树挂在了浅树下，后续的查找操作将会更耗时。

所以初始化时将每个元素的深度置为0，当合并两个连通子集时，将浅树的代表元的父元修改为深树的代表元，同时将浅树从深度记录中移除。特别地，如果深度一致，则随机选择，并且需要将树的深度增加1。

这样做的另一个好处是，可以很容易地从深度记录中知道当前连通子集的个数。缺点是增加了空间开销。

<div class="alert alert-block alert-warning">
    <i class="fa fa-sticky-note" aria-hidden="true"><b> Note:</b></i>
    并查集在查找元素代表元的时候会压缩路径，也即是说有可能会改变树深，而此时难以做到同步维护深度记录，所以经过一些查找后，树深记录已经不再可靠，该优化意义不大，建议不做。
</div>

In [14]:
from collections import defaultdict


class DisjointSet:
    """并查集"""
    def __init__(self, iters):
        """
        初始化，每个点的父元都赋为自身，深度都置为0
        :param iters: 可迭代对象 
        """
        self.father = {x: x for x in iters}
        self.depth = dict.fromkeys(iters, 0)
    
    def union(self, items):
        """
        将一簇点连通起来
        :param items:
        :return:
        """
        items = list(items)
        f = self.find(items[0])
        for item in items[1:]:
            itemf = self.find(item)
            if itemf == f: 
                continue
            if self.depth[itemf] > self.depth[f]:
                self.father[f] = itemf
                self.depth.pop(f)
                f = itemf
            else:
                if self.depth[itemf] == self.depth[f]:
                    self.depth[f] += 1
                self.father[itemf] = f
                self.depth.pop(itemf)

    def find(self, item):
        """
        查找代表元，同时进行路径压缩
        :param item:
        :return:
        """
        if self.father[item] != item:
            self.father[item] = self.find(self.father[item])  # 若 a -> b -> c -> 代表元x，则将a、b、c的父元均修改为x
        return self.father[item]
    
    def is_connected(self, u, v):
        """
        判断两个点是否连通
        :param u:
        :param v:
        :return:
        """
        return self.find(u) == self.find(v)
    
    def get_cluster(self, item=None):
        """
        获得指定元素的所在的连通子集，如无指定，则统计所有连通子集
        :return: 
        """
        if item is None:
            clusters = defaultdict(set)
            for item in self.father:
                clusters[self.find(item)].add(item)
            return clusters
        else:
            sets = set()
            itemf = self.find(item)
            for k in self.father:
                if self.find(k) == itemf:
                    sets.add(k)
            return sets
        
    def count(self):
        """返回连通子集个数"""
        return len(self.depth)

In [26]:
d = DisjointSet(range(13))
d.count()

13

In [27]:
sets = [{0, 1, 2}, {3, 4}, {5, 6, 7}, {8, 9, 10, 11}, {12}]
for items in sets:
    d.union(items)
d.count()

5

In [28]:
d.depth

{0: 1, 3: 1, 5: 1, 8: 1, 12: 0}

In [29]:
d.union([3, 5])
d.depth

{0: 1, 3: 2, 8: 1, 12: 0}

In [30]:
d.union([3, 8])
d.depth

{0: 1, 3: 2, 12: 0}

In [31]:
d.union([0, 3])
d.depth

{3: 2, 12: 0}

In [32]:
d.get_cluster()

defaultdict(set, {3: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}, 12: {12}})

In [33]:
d.get_cluster(7)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

In [34]:
d.is_connected(3, 12)

False

In [35]:
d.depth

{3: 2, 12: 0}

In [36]:
d.father

{0: 3,
 1: 3,
 2: 3,
 3: 3,
 4: 3,
 5: 3,
 6: 3,
 7: 3,
 8: 3,
 9: 3,
 10: 3,
 11: 3,
 12: 12}

### 去除了对全体的初始化
代码最为简洁，效率也没有提升。

In [42]:
from collections import defaultdict


class DisjointSet(dict):
    """并查集"""
    
    def find(self, item):
        """
        查找代表元，同时进行路径压缩
        :param item:
        :return:
        """
        father = self.setdefault(item, item)
        if father != item:
            self[item] = self.find(father)  # 若 a -> b -> c -> 代表元x，则将a、b、c的父元均修改为x
        return self[item]
    
    def union(self, items):
        """
        将一簇点连通起来
        :param items:
        :return:
        """
        items = list(items)
        f = self.find(items[0])
        for item in items[1:]:
            itemf = self.find(item)
            if itemf != f:  # 否则存在环
                self[itemf] = f 

    def is_connected(self, u, v):
        """
        判断两个点是否连通
        :param u:
        :param v:
        :return:
        """
        return self.find(u) == self.find(v)

    def get_cluster(self, item=None):
        """
        获得指定元素的所在的连通子集，如无指定，则统计所有连通子集
        :return: 
        """
        if item is None:
            clusters = defaultdict(set)
            for item in self:
                clusters[self.find(item)].add(item)
            return clusters
        else:
            sets = set()
            itemf = self.find(item)
            for k in self:
                if self.find(k) == itemf:
                    sets.add(k)
            return sets
    
    def count(self):
        """返回连通子集个数"""
        return len(self.get_cluster())

In [45]:
d = DisjointSet()
d
d.count() 

{}

0

In [46]:
sets = [{0, 1, 2}, {3, 4}, {5, 6, 7}, {8, 9, 10, 11}, {12}]
for items in sets:
    d.union(items)
d
d.count()

{0: 0,
 1: 0,
 2: 0,
 3: 3,
 4: 3,
 5: 5,
 6: 5,
 7: 5,
 8: 8,
 9: 8,
 10: 8,
 11: 8,
 12: 12}

5

In [47]:
d.find(11)

d.is_connected(0, 1)
d.is_connected(0, 3)

d.get_cluster()

d.get_cluster(2)

8

True

False

defaultdict(set,
            {0: {0, 1, 2},
             3: {3, 4},
             5: {5, 6, 7},
             8: {8, 9, 10, 11},
             12: {12}})

{0, 1, 2}

# 复杂度（待研究）
$n$次合并$m$次查找的时间复杂度为$O(m*\alpha(n))$，其中$\alpha$是Ackerman函数的某个反函数，在很大的范围内（人类目前观测到的宇宙范围估算有$10^{80}$个原子，这小于前面所说的范围）这个函数的值可以看成是不大于4的，所以并查集的操作可以看作是线性的。详细证明见 [DisjointSet.pdf](documents/DisjointSet.pdf)。

In [48]:
def ackerman(m, n):
    # 百度百科中的定义
    if (m, n) == (1, 0):
        return 2
    if m == 0:
        return 1
    if n == 0:
        return m + 2
    return ackerman(ackerman(m - 1, n), n - 1)

In [49]:
ackerman(1, 0)
ackerman(0, 10)
ackerman(10, 0)
ackerman(5, 1)

2

1

12

10

# 应用

## 维护无向图的连通性
- 判断两个点是否在同一连通子集内。
- 判断无向图中是否存在环。如果在给两个顶点添加边的时候，发现这两个顶点已经连通了，那么就存在环。

## 最小生成树 *Kruskal*算法

## 最近公共祖先 *LCA*

# 例题
1. [LeetCode 并查集](https://leetcode-cn.com/tag/union-find/)
2. [【LeetCode】并查集 union-find（共16题）](http://www.cnblogs.com/zhangwanying/p/9964303.html)

- **入门**
    - [How Many Tables](http://acm.hdu.edu.cn/showproblem.php?pid=1213)
    - [1232 畅通工程](http://acm.hdu.edu.cn/showproblem.php?pid=1232)
    - [LeetCode 547. 朋友圈](https://leetcode-cn.com/problems/friend-circles/)
- **中等**
    - [LeetCode 130. 被围绕的区域](https://leetcode-cn.com/problems/surrounded-regions)
    - [LeetCode 200. 岛屿的个数](https://leetcode-cn.com/problems/number-of-islands/)
    - LeetCode 305. 岛屿的个数 II（未解锁）：[[leetcode] 305. Number of Islands II 解题报告](https://blog.csdn.net/qq508618087/article/details/50985158)
    - [LeetCode 684. 冗余连接](https://leetcode-cn.com/problems/redundant-connection/)
    - [LeetCode 685. 冗余连接 II](https://leetcode-cn.com/problems/redundant-connection-ii/submissions/)
    - [LeetCode 721. 账户合并](https://leetcode-cn.com/problems/accounts-merge/)
    - [LeetCode 778. 水位上升的泳池中游泳](https://leetcode-cn.com/problems/swim-in-rising-water/)
- **困难**
    - [803. 打砖块](https://leetcode-cn.com/problems/bricks-falling-when-hit/submissions/)